In [8]:
#Baseline model for purchase prediction
from collections import defaultdict
import operator
def readJson(f):
    for l in open(f):
        yield eval(l)
### Purchasing baseline: just rank which items are popular and which are not, and return '1' if an item is among the top-ranked

itemCount = defaultdict(int)
totalPurchases = 0

for l in readJson('train.json'):
    user,item = l['reviewerID'],l['asin']
    itemCount[item] += 1
    totalPurchases += 1

mostPopular = [(itemCount[x], x) for x in itemCount]
# mostPopular.sort()
list1=sorted(mostPopular, key=lambda x: (-x[0]))

return1 = set()
count = 0
for ic, i in list1:
    count += ic
    return1.add(i)
    if count > totalPurchases/2: break

# predictions = open("predictions_Purchase.txt", 'w')
# for l in open("pairs_Purchase.txt"):
#     if l.startswith("reviewerID"):
#     #header
#         predictions.write(l)
#         continue
#     u,i = l.strip().split('-')
#     if i in return1:
#         predictions.write(u + '-' + i + ",1\n")
#     else:
#         predictions.write(u + '-' + i + ",0\n")
# predictions.close()


Report for Task2:
Started with clustering items depending on their category and rating values and see whether user is past have reviewed or purchased any items from this cluster . If yes then user is likely to review or buy this particualr item.
But not getting better results , tried graphlab package implementation to predict k items for given user -item data . Storing all items and comparing whether this particular item is in top 20000 recommendation for the user then it is likely that user will purchase or review else 0. Result was computationally extensive to calculate. Took 13 hours to get 50000 purchase prediction. So at the end took the count of each item for clustering as that is the most important parameter in predicting purchase. A user will buy particular item only if in past this item is purchased by many user and has rating also above average and as well forming clusters depending on both prameters along with category .
After getting cluster label for each item and merged to get list of cluster labels which user generally purchased. 
If this new item has cluster label common with user cluster labels then its 1 else 0.
* not keeping code of earlier approaches.


In [1]:
#getting the data from train.json and meta,
from collections import defaultdict
import pandas as pd
import numpy as np
import nltk
import re
import os
import codecs
from sklearn import feature_extraction
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import sklearn.metrics as metrics
from sklearn.metrics import pairwise_distances
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans


def readJson(f):
    for l in open(f):
        yield eval(l)

trainuser=[]
trainitem=[]
value=[]

allRatings=[]
userRatings=defaultdict(list)
itemRatings=defaultdict(list)

itemCount = defaultdict(int)
totalPurchases = 0

for l in readJson('train.json'):
    user,item=l['reviewerID'],l['asin']
    trainuser.append(l['reviewerID'])
    trainitem.append(l['asin'])
    value.append(l['overall'])
    userRatings[user].append(l['overall'])
    itemCount[item] +=1 
    totalPurchases += 1
    allRatings.append(l['overall'])
    itemRatings[item].append(l['overall'])

userAverage = {}
itemAverage = {}
countAverage={}

globalAverage = sum(allRatings) / len(allRatings)
globalcount=(1000000/171185)
# for u in itemCount:
#     countAverage = sum(itemCount[u]) / len(itemCount[u])
countAverage = [(itemCount[x],x) for x in itemCount]

for u in itemRatings:
    itemAverage[u] = sum(itemRatings[u]) / len(itemRatings[u])

for u in userRatings:
    userAverage[u] = sum(userRatings[u]) / len(userRatings[u])

mostPopular = [(itemCount[x], x) for x in itemCount]
# mostPopular.sort()
list1=sorted(mostPopular, key=lambda x: (-x[0]))
return1 = set()
count = 0
for ic, i in list1:
    count += ic
    return1.add(i)
    if count > totalPurchases/2: break

allitems=[]
category=[] 
price=[]
for m in readJson('meta.json'):
    allitems.append(m['asin'])
    category.append(m['categories'])
# #     print (m['price'])
#     if m['price'] == None  :
#         price.append("NA")
#     else:
#         price.append(m['price'])

category = map(lambda s: s.replace('[[',''), category)
category = map(lambda s: s.replace(']]',''), category)
category = map(lambda s: s.replace(",",""), category)
category = map(lambda s: s.replace("'",""), category)
category = map(lambda s: s.replace("Electronics",""), category)

traindata={"reviewerID":trainuser,"asin":trainitem,"overall":value}
traindf=pd.DataFrame(traindata)
# print traindf

userAverage={"userAverage":userAverage}
userAveragedf=pd.DataFrame(userAverage)
# print userAveragedf
userAveragedf['reviewerID'] = userAveragedf.index
# print userAveragedf
traindfavg=pd.merge(traindf, userAveragedf, on='reviewerID', how='left')
traindfavg['noroverall']=traindfavg['overall']-traindfavg['userAverage']
print traindfavg


itemdata={"asin":allitems,"category":category}
itemdf=pd.DataFrame(itemdata)
# print itemdf

itemAverage={"itemAverage":itemAverage}
itemAveragedf=pd.DataFrame(itemAverage)
# print itemAveragedf
itemAveragedf['asin'] = itemAveragedf.index
# print itemAveragedf
itemdfavg=pd.merge(itemdf, itemAveragedf, on='asin', how='left')
itemdfavg=itemdfavg.fillna(globalAverage)
print itemdfavg


                                                     asin  overall  \
0       a6ed402934e3c1138111dce09256538afb04c566edf37c...      2.0   
1       8939d791e9dd035aa58da024ace69b20d651cea4adf615...      4.0   
2       8939d791e9dd035aa58da024ace69b20d651cea4adf615...      2.0   
3       8939d791e9dd035aa58da024ace69b20d651cea4adf615...      4.0   
4       8939d791e9dd035aa58da024ace69b20d651cea4adf615...      1.0   
5       6ac521636646aaecf483e148a7240f6214b7e5dc809400...      1.0   
6       5da57c87204216c7df1b6d40979a1067f2d0247291cc2b...      4.0   
7       59249d92675e68e4e635c03267aba9e58fa593fd6452ce...      4.0   
8       59249d92675e68e4e635c03267aba9e58fa593fd6452ce...      5.0   
9       59249d92675e68e4e635c03267aba9e58fa593fd6452ce...      5.0   
10      59249d92675e68e4e635c03267aba9e58fa593fd6452ce...      4.0   
11      20452efd72b6b22b093850a8d263c08e0b472edb3300b3...      3.0   
12      20452efd72b6b22b093850a8d263c08e0b472edb3300b3...      5.0   
13      ae42d701ca34

In [2]:
# print type(countAverage)
# print countAverage[0]
countAveragedf=pd.DataFrame(countAverage, columns=('count','asin'))
# print countAveragedf

#get count for each item into dataframe and merge it with train.json data
countdfavg=pd.merge(itemdfavg, countAveragedf, on='asin', how='left')
countdfavg=countdfavg.fillna(globalcount)
print countdfavg



                                                     asin  \
0       ed2538e73bd60a71a3cdb5c1d7e1558fa04aa05c4856a5...   
1       805e2f92a47fa1949b03bb6eb623ba85c58fb7f252042d...   
2       24a01893dbd46c9c695b53d671fb2c461e131b1284f932...   
3       a6ed402934e3c1138111dce09256538afb04c566edf37c...   
4       8939d791e9dd035aa58da024ace69b20d651cea4adf615...   
5       ba65694f68e1385fc1f5d734747c7e95e7f122ddeff40c...   
6       6ac521636646aaecf483e148a7240f6214b7e5dc809400...   
7       d249f7b6aa564a07913dfb397aecd67b0c1b1ae8031c86...   
8       9fbcf079e4dba8e3ba33132fff2b523dc24ed38d9a4c48...   
9       fa1b01d93f61eafbdd4388b3cda5ef49572280fd163ed3...   
10      78cb1c8c60232b6d9e64bd00d8c3cd7cb3a6d8c8fdbc7b...   
11      289eef8ad63ecf5b8ed22e870bbbc0d8dbf5da64b4a9db...   
12      7a49de7024c2279e0b92ffaa6e4c783da87883dbbd8053...   
13      3473195e17b9c7023bada8063f27c618bae7842e655072...   
14      fbcf9373993e379908f1f7b68d09779766fdee6e963faf...   
15      ae42d701ca34d0d3

In [4]:
#Prepare vector for clustering (on category of items, count of reviews of items and Average rating of item)
from sklearn.preprocessing import StandardScaler
vectorizer = TfidfVectorizer(min_df=4, max_df=0.8)
catvec=vectorizer.fit_transform(category)
# print catvec
catarr=catvec.toarray()


# standard scale itemAverage and count                      
ss=StandardScaler()
avgvec= ss.fit_transform(countdfavg['itemAverage'])
countvec= ss.fit_transform(countdfavg['count'])

# print avgvec
# print("-----")
# print countvec
#stacking all vectors together for k mean clustering
finalarr = (np.vstack((avgvec,countvec,catarr.T))).T
# print finalarr

/Users/vsanghvi007/anaconda/envs/dato-env/lib/python2.7/site-packages/sklearn/preprocessing/data.py:583: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/Users/vsanghvi007/anaconda/envs/dato-env/lib/python2.7/site-packages/sklearn/preprocessing/data.py:646: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/Users/vsanghvi007/anaconda/envs/dato-env/lib/python2.7/site-packages/sklearn/preprocessing/data.py:583: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.

In [5]:
#kmeans clustering 
kmeans = KMeans(init='k-means++', n_clusters=50, n_init=10)
kmeans.fit_predict(finalarr)
centroids = kmeans.cluster_centers_
labels = kmeans.labels_
error = kmeans.inertia_
#print kmeans clustering  clusters ie labels and centroids
print "The total error of the clustering is: ", error
print '\nCluster labels'
print labels
print '\n Cluster Centroids'
print centroids


The total error of the clustering is:  294379.2058

Cluster labels
[27 27 27 ..., 21 21 45]

 Cluster Centroids
[[  1.65011380e+00  -3.16822650e-01   5.92923063e-19 ...,   2.91379334e-18
    1.92445886e-18   3.61310374e-17]
 [  1.01012542e+00  -3.36160904e-02   9.13101517e-19 ...,   8.59919668e-05
    4.22495420e-04   2.04981099e-04]
 [ -2.67814353e+00  -2.93845555e-01   2.16248741e-05 ...,   4.89825210e-05
    1.44469939e-17   2.26565098e-04]
 ..., 
 [  3.63690000e-02   2.20296179e+01   1.10114283e-20 ...,   4.74338450e-20
    1.62630326e-19  -1.62630326e-19]
 [  6.44047809e-02  -4.92775709e-02  -3.54059772e-19 ...,   4.78404209e-18
    2.23345648e-17   2.39337630e-17]
 [  3.65077122e-02  -2.59339771e-02  -2.96461532e-20 ...,   4.16740210e-18
    8.99568699e-03   9.64939934e-18]]


In [14]:
#calculating the cluster label and merging it with train.json data
import scipy.spatial.distance as dist
def clust_mean(grp):
    grp['ClusterAverage'] = grp['itemAverage'].mean()
    return grp
  
countdfavg['cluster']=labels
# print countdfavg
clusterdf=countdfavg.groupby(['cluster']).apply(clust_mean)


userdf=pd.merge(traindfavg, clusterdf, on='asin', how='left')
print userdf

#write the dictionary for future use
userdict={}
clusterdict= dict(zip(clusterdf['asin'], clusterdf['cluster']))
# userdict=dict(zip(userdf['reviewerID'],userdf['cluster']))
for i, rows in userdf.iterrows():
    if rows['reviewerID'] not in userdict:
        userdict[rows['reviewerID']] = {rows['cluster']}
    else:
        userdict[rows['reviewerID']].update({rows['cluster']})


                                                     asin  overall  \
0       a6ed402934e3c1138111dce09256538afb04c566edf37c...      2.0   
1       8939d791e9dd035aa58da024ace69b20d651cea4adf615...      4.0   
2       8939d791e9dd035aa58da024ace69b20d651cea4adf615...      2.0   
3       8939d791e9dd035aa58da024ace69b20d651cea4adf615...      4.0   
4       8939d791e9dd035aa58da024ace69b20d651cea4adf615...      1.0   
5       6ac521636646aaecf483e148a7240f6214b7e5dc809400...      1.0   
6       5da57c87204216c7df1b6d40979a1067f2d0247291cc2b...      4.0   
7       59249d92675e68e4e635c03267aba9e58fa593fd6452ce...      4.0   
8       59249d92675e68e4e635c03267aba9e58fa593fd6452ce...      5.0   
9       59249d92675e68e4e635c03267aba9e58fa593fd6452ce...      5.0   
10      59249d92675e68e4e635c03267aba9e58fa593fd6452ce...      4.0   
11      20452efd72b6b22b093850a8d263c08e0b472edb3300b3...      3.0   
12      20452efd72b6b22b093850a8d263c08e0b472edb3300b3...      5.0   
13      ae42d701ca34

In [16]:
#just for validation purpose
u = "a81d82779006ee17e589c6a56e20102694094b85789732aa1ee6643c96b51558"
if u in userdict:
    print userdict[u] 
    print type(userdict[u])

set([10, 3, 36, 37, 47])
<type 'set'>


In [18]:
#writing prediction as 1 if item cluster label is in user cluster labels else 0. Also for cold start scenarios, use the most popular item logic to predict 1 else 0.
predictions = open("predictions_Purchase.txt", 'w')
for l in open("pairs_Purchase.txt"):
    if l.startswith("reviewerID"):
    #header
        predictions.write(l)
        continue
    u,i = l.strip().split('-')
    if u in userdict:
        if clusterdict[i] in userdict[u]:
            predictions.write(u + '-' + i + ",1\n")
        else:
            predictions.write(u + '-' + i + ",0\n")
    else:
        if i in return1:
            predictions.write(u + '-' + i + ",1\n")
        else:
            predictions.write(u + '-' + i + ",0\n")
predictions.close()